In [ ]:
from keras.applications.convnext import LayerScale
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow import keras

from data_preparation import prepare_data
from model_preprocessing import convert_image_to_array, model_predict
from grad_cam import make_gradcam_heatmap

## Load data

In [ ]:
TARGET_HEIGHT = 640
TARGET_WIDTH = 640

In [ ]:
X_val, y_val, valid_labels_df, targets = prepare_data()

## Load model

In [ ]:
MODEL_NAME = 'pretrain_model_ConvNeXtBase_w_ClssWgt_01-0.3616.h5'
model_path = f'../models/{MODEL_NAME}'
model = tf.keras.models.load_model(model_path, custom_objects={'LayerScale': LayerScale})

In [ ]:
model.summary()

## Define Grad-CAM Algorithm

In [ ]:
MY_IMG_PATH = X_val[6]

In [ ]:
MY_IMG_PATH

In [ ]:
y_val[6]

In [ ]:
last_conv_layer_name = 'block14_sepconv2'
classifier_layer_names = ['local_avg_pool', 'flatten', 'prediction']

Run model on a single image

In [ ]:
test_num = 9

X_val_example = X_val[test_num]
y_val_example = y_val[test_num]

y_hat = model_predict(path=X_val_example, model=model, target_height=TARGET_HEIGHT, target_width=TARGET_WIDTH)

print(f'Ground truth label: {y_val_example} \n Predicted label: {np.argmax(y_hat)} \t Probability: {np.max(y_hat)}')

In [ ]:
y_hat

In [ ]:
def get_img_array(img_path):
    global TARGET_HEIGHT
    global TARGET_WIDTH
    img = convert_image_to_array(path=img_path, target_height=TARGET_HEIGHT, target_width=TARGET_WIDTH)
    # We add a dimension to transform our array into a "batch"
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
# Prepare image
img_array = get_img_array(img_path=MY_IMG_PATH)

# Print what the top predicted class is
preds = model.predict(img_array)
pred_class = np.argmax(preds)
pred_prob = np.max(preds)
#print(f"Predicted class: {pred_label} \n Probability: {pred_prob} \n Actual Class: {lesion_type_dict[label_abbreviation]}")
print(f"Predicted class: {pred_class} \n Probability: {pred_prob}")

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
# We load the original image
img = convert_image_to_array(path=MY_IMG_PATH, target_height=TARGET_HEIGHT, target_width=TARGET_WIDTH)
img = np.uint8(255. * img)

# We rescale heatmap to a range 0-255
heatmap = np.uint8(255. * heatmap)

# We use jet colormap to colorize heatmap
jet = cm.get_cmap("jet")

# We use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# We create an image with RGB colorized heatmap
jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

# Superimpose the heatmap on original image
superimposed_img = jet_heatmap * 0.2 + img
superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

# Save the superimposed image
save_path = "heatmap.jpg"
superimposed_img.save(save_path)

# Display Grad CAM along with original image
fig, axs = plt.subplots(1, 2, figsize = (10, 5))

axs[0].imshow(np.asarray(Image.open(save_path)))
axs[0].axis('off')
axs[0].set_aspect('auto')
axs[1].imshow(np.asarray(img))
axs[1].axis('off')
axs[1].set_aspect('auto')

plt.show() 